This notebook serves to get additional data from the EDH XML/epidoc files. The reason is that some information is missing in the API data.


In [1]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET
import re

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import sddk

# google sheets integration:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library


In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
try:
    file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
except:
    print("cannot find file ServiceAccountsKey.json")
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
EDH_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/164MLxVcCZg95Bzf9fVyD1-iCA5V97eM3KAFllyhTvt4/edit?usp=sharing")

Now we turn to the download section of the EDH website, where we can find zip archives containing xml files with individual inscriptions. Instead of downloading them manually, we will download them directly into our Python environment.

In [4]:
# extract the download page
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/data/export", headers={"User-Agent" : ""})
url_text = resp.text

In [5]:
# extract urls of individual zip archives for download
download_urls = re.findall("download\/edhEpidocDump_HD.+", url_text)
download_urls

['download/edhEpidocDump_HD000001-HD010000.zip',
 'download/edhEpidocDump_HD010001-HD020000.zip',
 'download/edhEpidocDump_HD020001-HD030000.zip',
 'download/edhEpidocDump_HD030001-HD040000.zip',
 'download/edhEpidocDump_HD040001-HD050000.zip',
 'download/edhEpidocDump_HD050001-HD060000.zip',
 'download/edhEpidocDump_HD060001-HD070000.zip',
 'download/edhEpidocDump_HD070001-HD082046.zip']

In [7]:
# collecting filenames for testing

url_base = "https://edh-www.adw.uni-heidelberg.de/"

url = url_base + download_urls[0]
resp = requests.get(url, headers={"User-Agent" : ""})
zipped = zipfile.ZipFile(io.BytesIO(resp.content))
### names of all files within the zipped directory
namelist = zipped.namelist()

In [6]:
# define function for data parsing
def get_data_from_filename(filename, zipped):
    try:
        soup = BeautifulSoup(zipped.read(filename))
        xml_data = {} 
        idno_uri = soup.find("idno", attrs={"type" : "URI"}).get_text()
        xml_data["idno_uri"] = idno_uri.rpartition("/")[2]
        xml_data["idno_tm"] = soup.find("idno", attrs={"type" : "TM"}).get_text()
        placenames_refs = []
        try: 
            placenames = soup.find_all("placename")
            for placename in placenames:
                placenames_refs.append(placename["ref"])
        except: placenames_refs = []
        xml_data["placenames_refs"] = placenames_refs
        text_tag = soup.find("div", attrs={"type" : "edition"})
        xml_data["text_edition"] = " ".join(text_tag.get_text().splitlines()[1:])
        xml_data["origdate_text"] = soup.find("origdate").get_text().replace("\n", "")
        try: 
            layout_execution = soup.layout.find("rs")["ref"]
            xml_data["layout_execution"] = layout_execution.rpartition("/")[2]
        except: xml_data["layout_execution"] = ""
        try: xml_data["layout_execution_text"] = soup.layout.rs.get_text()
        except: xml_data["layout_execution_text"] = ""
        try: 
            support_objecttype = soup.support.find("objecttype")["ref"]
            xml_data["support_objecttype"] = support_objecttype.rpartition("/")[2]
        except: xml_data["support_objecttype"] = ""
        try: xml_data ["support_objecttype_text"] = soup.support.objecttype.get_text()
        except: xml_data ["support_objecttype_text"] = ""
        try: 
            support_material = soup.support.find("material")["ref"]
            xml_data["support_material"] = support_material.rpartition("/")[2]
        except: xml_data["support_material"] = ""    
        try: xml_data["support_material_text"] = soup.support.material.get_text()
        except: xml_data["support_material_text"] = ""
        try: 
            support_decoration = soup.support.find("rs")["ref"]
            xml_data["support_decoration"] = support_decoration.rpartition("/")[2]
        except: xml_data["support_decoration"] = ""
        try: 
            keywords_term = soup.keywords.find("term")["ref"]
            xml_data["keywords_term"] = keywords_term.rpartition("/")[2]
        except: xml_data["keywords_term"] = ""
        try: xml_data["keywords_term_text"] = soup.keywords.get_text().replace("\n", "")
        except: xml_data["keywords_term_text"] = ""
        return xml_data
    except:
        pass

In [9]:
# test with first ten files within the namelist
edh_xml_data = []

for filename in namelist[:10]:
    edh_xml_data.append(get_data_from_filename(filename, zipped))
# transform it into dataframe
pd.DataFrame(edh_xml_data)

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,HD000001,251193,"[http://www.trismegistos.org/place/033152, htt...",Dis Manibus Noniae Publi filiae Optatae et Cai...,71 AD – 130 AD,21,unbestimmt,257,Tafel,,"Marmor, geädert / farbig",1000,92,Grabinschrift
1,HD000002,265631,"[http://www.trismegistos.org/place/000172, htt...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
2,HD000003,220675,"[http://www.trismegistos.org/place/025443, htt...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
3,HD000004,222102,"[http://www.trismegistos.org/place/025443, htt...",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
4,HD000005,265629,"[http://www.trismegistos.org/place/000172, htt...",libertus Successus Luci libertus Irenaeus C...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift
5,HD000006,222924,"[http://www.trismegistos.org/place/025443, htt...",Dis Manibus sacrum Memmia Auctina annorum LXX...,71 AD – 150 AD,21,unbestimmt,250,Stele,60,Kalkstein,1000,92,Grabinschrift
6,HD000007,265588,"[http://www.trismegistos.org/place/000172, htt...",Clodia Marci filia,100 BC – 51 BC,21,unbestimmt,257,Tafel,71,Travertin,1000,92,Grabinschrift
7,HD000008,265611,"[http://www.trismegistos.org/place/000172, htt...",Dis Manibus Caio Satrio Xantho Cai Satri Rufi ...,101 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
8,HD000009,168722,[],ABCDEFX,201 AD – 300 AD,21,unbestimmt,276,Tessera,108,"Blei, Zinn",1000,76,Defixio
9,HD000010,244297,[],Dis Manibus Luci Asini Poli Secundus et Orphae...,101 AD – 200 AD,21,unbestimmt,78,Urne,138,unbestimmt,1000,92,Grabinschrift


In [10]:
EDH_xml_cols = pd.DataFrame(pd.DataFrame(edh_xml_data).columns, columns=["columns"])
EDH_xml_cols

,columns
0,idno_uri
1,idno_tm
2,placenames_refs
3,text_edition
4,origdate_text
5,layout_execution
6,layout_execution_text
7,support_objecttype
8,support_objecttype_text
9,support_material


In [11]:
# uncomment to export to gsheets
# set_with_dataframe(EDH_overview.add_worksheet("EDH_xml_cols", 1, 1), EDH_xml_cols)

In [7]:
%%time

# main loop

url_base = "https://edh-www.adw.uni-heidelberg.de/"
edh_xml_data = []

for d_url in download_urls:
    url = url_base + d_url
    print(url)
    resp = requests.get(url, headers={'User-Agent': ''})
    zipped = zipfile.ZipFile(io.BytesIO(resp.content))
    ### names of all files within the zipped directory
    namelist = zipped.namelist()
    for filename in namelist:
        try:
            edh_xml_data.append(get_data_from_filename(filename, zipped))
        except:
            pass
        ### index "0" is for main directory

https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD000001-HD010000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD010001-HD020000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD020001-HD030000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD030001-HD040000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD040001-HD050000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD050001-HD060000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD060001-HD070000.zip
https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD070001-HD082046.zip
CPU times: user 12min 5s, sys: 6.05 s, total: 12min 11s
Wall time: 30min 19s


In [8]:
# remove empty
#edh_xml_data = [elem for elem in edh_xml_data if elem != None]
# how many we have
# last time we had 81143
len(edh_xml_data)

81156

In [9]:
edh_xml_data_f = [el for el in edh_xml_data if el != None]

In [10]:
len(edh_xml_data_f)

81151

In [11]:
# make a dataframe from 
edh_xml_data_df = pd.DataFrame(edh_xml_data_f)
edh_xml_data_df.head(5)

,idno_uri,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text
0,HD000001,251193,"[http://www.trismegistos.org/place/033152, htt...",Dis Manibus Noniae Publi filiae Optatae et Cai...,71 AD – 130 AD,21,unbestimmt,257,Tafel,,"Marmor, geädert / farbig",1000,92,Grabinschrift
1,HD000002,265631,"[http://www.trismegistos.org/place/000172, htt...",Caius Sextius Paris qui vixit annis LXX ...,51 AD – 200 AD,21,unbestimmt,257,Tafel,48,Marmor,1000,92,Grabinschrift
2,HD000003,220675,"[http://www.trismegistos.org/place/025443, htt...",Publio Mummio Publi filio Galeria Sisennae Rut...,131 AD – 170 AD,21,unbestimmt,57,Statuenbasis,48,Marmor,1000,69,Ehreninschrift
3,HD000004,222102,"[http://www.trismegistos.org/place/025443, htt...",AVSLLA Marci Porci Nigri serva dominae Veneri ...,151 AD – 200 AD,21,unbestimmt,29,Altar,60,Kalkstein,1000,80,Weihinschrift
4,HD000005,265629,"[http://www.trismegistos.org/place/000172, htt...",libertus Successus Luci libertus Irenaeus C...,1 AD – 200 AD,21,unbestimmt,250,Stele,138,unbestimmt,1000,92,Grabinschrift


In [12]:
sddk.write_file("SDAM_data/EDH/edh_xml_data_2020-09-22.json", edh_xml_data_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/edh_xml_data_2020-09-22.json"


# Read the data back

In [3]:
edh_xml_data_df = sddk.read_file("SDAM_data/EDH/edh_xml_data_2020-09-22.json", "df", conf)